# StanceEval

In [110]:
from pprint import pprint
from sklearn import metrics
import numpy as np
import sys

def _metrics_report_to_df(ytrue, ypred):
    precision, recall, fscore, support = metrics.precision_recall_fscore_support(ytrue, ypred)
    acc = metrics.accuracy_score(ytrue, ypred)
    classification_report = pd.concat(map(pd.DataFrame, [[acc,acc], fscore, precision, recall, support]), axis=1)
    classification_report.columns = ["accuracy", "f1-score", "precision", "recall", "support"]
    return(classification_report)

import pandas as pd
# data="kwon/sentiments.txt"
data="twitter-ma/twitter-ma.csv"

df = pd.read_csv(data)
print df.head()
print df.columns
# sys.exit()

senti = df.groupby(['fact'], as_index=False).mean()
label = (senti['sentiment_score']>0)*1

print senti.head()
print senti.columns
print len(label)

ground = []
for row in senti['fact']:
    if row.split("_")[0] == 'N':
        ground.append(0)
    else:
        ground.append(1)
print len(ground)
        
report = _metrics_report_to_df(ground, label) # metrics.classification_report(labels, assigned, target_names=self.keys, output_dict=True)
pprint(report)       

# Kwon

# accuracy  f1-score  precision    recall  support
# 0  0.531532  0.527273   0.491525  0.568627       51
# 1  0.531532  0.535714   0.576923  0.500000       60

# MA
# accuracy  f1-score  precision    recall  support
# 0  0.486218  0.572477   0.474886  0.720554      433
# 1  0.486218  0.356354   0.516000  0.272152      474


      website    fact  object  sentiment_score
0  1003856029  N_E312  N_E312          -0.6597
1  1013430282  N_E312  N_E312          -0.7149
2  1014445069  N_E312  N_E312          -0.7149
3   101596871  N_E312  N_E312           0.5267
4    10228272  N_E312  N_E312          -0.7412
Index([u'website', u'fact', u'object', u'sentiment_score'], dtype='object')
                       fact       website  sentiment_score
0               N_Airfrance  2.313271e+07        -0.313637
1                N_Airliner  1.396921e+07         0.129432
2                  N_Amanda  2.438650e+07        -0.103014
3                 N_AnnieLe  4.059135e+07        -0.194677
4  N_BarnesNobleObamaMonkey  1.760870e+07         0.238274
Index([u'fact', u'website', u'sentiment_score'], dtype='object')
992
992
   accuracy  f1-score  precision    recall  support
0  0.469758  0.532028   0.474603  0.605263      494
1  0.469758  0.388372   0.461326  0.335341      498


# KMeans

In [33]:
# Number of posts in the cascade
# Time: Total time length of cascade, Avg time gap between posts in cascade
## ignore tree: Avg Depth of tree, Avg Breadth of tree 
 
# data="twitter-ma/train_cascades.txt"
# ground_data="twitter-ma/train_labels.txt"
# data="train_cascades.txt"
# ground_data="train_labels.txt"

# data="../data/kwon/cascades.txt"
# ground_data="../data/kwon/labels.txt"

data="../data/tma/cascades.txt"
ground_data="../data/tma/labels.txt"


import numpy as np
import operator, sys
from sklearn import preprocessing

def extract_feats(ulist, tlist, top_users_set):
    total_time = tlist[-1]
    num_posts = len(ulist)
    time_gap = tlist[1:] - tlist[0:-1]
    frac_top_users = len(set(ulist) & top_users_set)*1.0/len(ulist)
    return [num_posts, total_time, np.mean(time_gap), frac_top_users]
    
    # feats = extract_feats(np.array([1,2,3]), np.array([2,5,6]))


cascades = []
upart = {}

tot_eng = 0
for line in open(data):
    # print(line)
    activations = line.strip().split(",")
    tot_eng += len(activations)/2
    ulist, tlist = [], []
    
    users = activations[0::2]
    times = activations[1::2]
    
    for u, t in zip(users, times):
        u = int(u)
        t = float(t)
        if u not in upart:
            upart[u] = 1
        else: upart[u] += 1
        ulist.append(u); tlist.append(float(t))    
    cascades.append((ulist, tlist))
    # break
print("user", len(upart), tot_eng)
# sys.exit()
    
sorted_x = sorted(upart.items(), key=operator.itemgetter(1), reverse=True)
top_users = sorted_x[:5000]
top_users_set = set()
for u, count in top_users:
    top_users_set.add(u)

X = []
for ulist, tlist in cascades:
    cas_features = extract_feats(np.array(ulist), np.array(tlist), top_users_set)
    X.append(cas_features)

X = np.array(X)
print(X.shape)
score = np.mean(X, axis=0)
score[1] = score[1]/3600.0; score[2] = score[2]/3600.0 
print(list(score)) # [num_posts, total_time, np.mean(time_gap), frac_top_users]
# sys.exit()
# print X

from sklearn.cluster import KMeans
from sklearn import metrics
import numpy as np

kmeans = KMeans(n_clusters=2, random_state=None).fit(preprocessing.normalize(X))
# kmeans = KMeans(n_clusters=2, random_state=None).fit(X)
assigned = kmeans.labels_# array([0, 0, 0, 1, 1, 1], dtype=int32)
# print assigned
print(len(assigned))

ground = np.loadtxt(ground_data)
report = metrics.classification_report(ground, assigned) # metrics.classification_report(labels, assigned, target_names=self.keys, output_dict=True)
print(report)
report = metrics.classification_report(ground, 1-assigned) # metrics.classification_report(labels, assigned, target_names=self.keys, output_dict=True)
print(report)
# print kmeans.predict([[0, 0], [4, 4]]) # array([0, 1], dtype=int32)
# print kmeans.cluster_centers_ # array([[1., 2.], [4., 2.]])

user 233719 592391.0
(992, 4)
[597.1683467741935, 1982.9470718525986, 64.61494570103345, 0.16573996807935304]
992
              precision    recall  f1-score   support

         0.0       0.51      0.25      0.33       494
         1.0       0.51      0.77      0.61       498

    accuracy                           0.51       992
   macro avg       0.51      0.51      0.47       992
weighted avg       0.51      0.51      0.47       992

              precision    recall  f1-score   support

         0.0       0.49      0.75      0.60       494
         1.0       0.49      0.23      0.31       498

    accuracy                           0.49       992
   macro avg       0.49      0.49      0.45       992
weighted avg       0.49      0.49      0.45       992



In [34]:
report = metrics.classification_report(ground, 1 - assigned)
print(report)
np.savetxt('../output/baselines/tma/kmeans_pred_results.txt', 1 - assigned, fmt='%d')

              precision    recall  f1-score   support

         0.0       0.49      0.75      0.60       494
         1.0       0.49      0.23      0.31       498

    accuracy                           0.49       992
   macro avg       0.49      0.49      0.45       992
weighted avg       0.49      0.49      0.45       992



# TF predictions fix

In [4]:
import pickle as pkl
import pandas as pd
   

datadir = 'kwon'
labels = '../data/kwon/labels.txt'
cascade_names = '../data/kwon/cascade_names.txt'
save_file = '../output/baselines/kwon/tf_pred_results.txt'

# datadir = 'tma'
# labels = '../data/tma/labels.txt'
# cascade_names = '../data/tma/cascade_names.txt'
# save_file = '../output/baselines/tma/tf_pred_results.txt'

dict_preds = {}

if datadir == 'kwon':
    r = pkl.load(open('../clustering_baselines/tf_kwon_results.pkl', 'rb'))
    scores = r['dataframe']['trustworthiness']
    events = r['dataframe']['fact']
    for event, score in zip(events, scores):
        # event, score = line.split(',')[0].strip(), float(line.split(',')[1].strip().strip('[]'))
        dict_preds[event] = (score < 0)*1
        # break
elif datadir == 'tma':
    f = open('../output/baselines/tma/tf_predictions.txt', 'r')
    for line in f.readlines():
        if line == '' or line == '\n':
            continue
        # print(line)
        event, score = line.split(',')[0].strip(), float(line.split(',')[1].strip().strip('[]'))
        # print(event, score)
        dict_preds[event] = (score < 0)*1
        # break
    # print(scores)
print(list(dict_preds.keys())[0])
dict_preds['N_Vince']

N_Vince


0

In [6]:
import re
from sklearn import metrics
import numpy as np

cnames = np.loadtxt(cascade_names, dtype=np.str)
gt = np.loadtxt(labels)
# print(cnames)
# # print(gt)

tar = []
preds = []
count0 = 0
for i, name in enumerate(cnames):
    n = re.sub('.txt$', '', name)
    if n in dict_preds:
        p = dict_preds[n]
        preds.append(p)
    else:
        preds.append(0)
        count0 += 1

print(count0)
print(metrics.classification_report(gt, preds))

np.savetxt(save_file, preds, fmt='%d')

0
              precision    recall  f1-score   support

         0.0       0.48      0.82      0.61        51
         1.0       0.62      0.25      0.36        60

    accuracy                           0.51       111
   macro avg       0.55      0.54      0.48       111
weighted avg       0.56      0.51      0.47       111

